In [1]:
!python -V
import gc as garbage_colector

Python 3.10.4


# <center> **ITERABLE**</center>
# <center> TIENEN EL METODO **.\__\___iter\__\___()**</center>

In [2]:
def is_palindrome(num):
    
    r = str(abs(num)) if type(int) else num

    if len(r) < 2:
        return False
    
    elif len(r) %2 == 0:
        size = len(r)//2
        a = r[:size]
        b = r[size:][::-1]
        
        return a==b        

    else:
        size = len(r)//2
        a = r[:size]
        b = r[size+1:][::-1]
        
        return a==b
    
listado = list()

for i in range(-1000,1000):

    if (is_palindrome(i)):
        listado.append(i)

print(listado)
print()
print(dir(listado))
print()

for valor in listado[::-1][0:7]:
    print(valor)  
print()

for valor in listado[::-1][0:7]:
    print(valor)
print()

for valor in listado[::-1][0:7]:
    print(valor)
print()

for valor in listado[::-1][0:7]:
    print(valor)
print()

[-999, -989, -979, -969, -959, -949, -939, -929, -919, -909, -898, -888, -878, -868, -858, -848, -838, -828, -818, -808, -797, -787, -777, -767, -757, -747, -737, -727, -717, -707, -696, -686, -676, -666, -656, -646, -636, -626, -616, -606, -595, -585, -575, -565, -555, -545, -535, -525, -515, -505, -494, -484, -474, -464, -454, -444, -434, -424, -414, -404, -393, -383, -373, -363, -353, -343, -333, -323, -313, -303, -292, -282, -272, -262, -252, -242, -232, -222, -212, -202, -191, -181, -171, -161, -151, -141, -131, -121, -111, -101, -99, -88, -77, -66, -55, -44, -33, -22, -11, 11, 22, 33, 44, 55, 66, 77, 88, 99, 101, 111, 121, 131, 141, 151, 161, 171, 181, 191, 202, 212, 222, 232, 242, 252, 262, 272, 282, 292, 303, 313, 323, 333, 343, 353, 363, 373, 383, 393, 404, 414, 424, 434, 444, 454, 464, 474, 484, 494, 505, 515, 525, 535, 545, 555, 565, 575, 585, 595, 606, 616, 626, 636, 646, 656, 666, 676, 686, 696, 707, 717, 727, 737, 747, 757, 767, 777, 787, 797, 808, 818, 828, 838, 848, 858

# <center> ITERADORES</center>
## <center> TIENEN LOS METODOS **.\__\___next\__\___() y .\__\___iter\__\___()** Y SE RECORREN CON LA FUNCION **next()** o el  metodo de la clase **.\__\___next\__\___()**</center>

In [3]:
from types import GeneratorType

tipos_de_dato_iterables = [set, tuple, dict, range, str, enumerate, GeneratorType, int, float, complex, bool]

for tipo_de_dato in tipos_de_dato_iterables:

    if '__iter__' in set(dir(tipo_de_dato)) and '__next__' in set(dir(tipo_de_dato)):
        print(f'la {tipo_de_dato} contiene el metodo __iter__ y el metodo __next__, es un ITERABLE y un ITERADOR')
        
    elif '__iter__' in set(dir(tipo_de_dato)):
        print(f'la {tipo_de_dato} contiene el metodo __iter__, es un ITERABLE ')
    else:
        print('*************************************************')
        print(f'la {tipo_de_dato} no es una estructura Iterable')



la <class 'set'> contiene el metodo __iter__, es un ITERABLE 
la <class 'tuple'> contiene el metodo __iter__, es un ITERABLE 
la <class 'dict'> contiene el metodo __iter__, es un ITERABLE 
la <class 'range'> contiene el metodo __iter__, es un ITERABLE 
la <class 'str'> contiene el metodo __iter__, es un ITERABLE 
la <class 'enumerate'> contiene el metodo __iter__ y el metodo __next__, es un ITERABLE y un ITERADOR
la <class 'generator'> contiene el metodo __iter__ y el metodo __next__, es un ITERABLE y un ITERADOR
*************************************************
la <class 'int'> no es una estructura Iterable
*************************************************
la <class 'float'> no es una estructura Iterable
*************************************************
la <class 'complex'> no es una estructura Iterable
*************************************************
la <class 'bool'> no es una estructura Iterable


<center>
    <img width="80%" lign="center" src="iterador.png">
</center>

In [4]:
listado = [1,2,3,4,5,6,7,8,9,0]

listado.__next__()

AttributeError: 'list' object has no attribute '__next__'

In [5]:
next(listado)

TypeError: 'list' object is not an iterator

In [6]:
listado.__iter__()

In [7]:
print(type(listado))
del listado
garbage_colector.collect(), garbage_colector.get_threshold()

<class 'list'>


(925, (700, 10, 10))

## <center> ITERABLE CONVERTIDO EN ITERADOR</center>
# <center> iter()</center>
# <center> el método __iter__ de una clase “Iterable” devuelve un “Iterador”, pero la clase “Iterador” hereda de “Iterable”.</center>

In [8]:
lista = [1,2,3,4,5,6,7,8,9,0]
print(dir(lista))

['__add__', '__class__', '__class_getitem__', '__contains__', '__delattr__', '__delitem__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__gt__', '__hash__', '__iadd__', '__imul__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__mul__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__reversed__', '__rmul__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__subclasshook__', 'append', 'clear', 'copy', 'count', 'extend', 'index', 'insert', 'pop', 'remove', 'reverse', 'sort']


# <center> CASTING CON iter()</center>

In [9]:
iterador_casting = iter(lista)
print(dir(iterador_casting))

['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__length_hint__', '__lt__', '__ne__', '__new__', '__next__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__']


In [10]:
print(type(iterador_casting))

print(iterador_casting.__next__())
print(iterador_casting.__next__())

print(next(iterador_casting))
print(next(iterador_casting))

del iterador_casting
garbage_colector.collect(), garbage_colector.get_threshold()

<class 'list_iterator'>
1
2
3
4


(0, (700, 10, 10))

# <center> .\__\___iter\__\___()</center>

In [11]:
iterador_metodo = lista.__iter__()
print(dir(iterador_metodo))

['__class__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__length_hint__', '__lt__', '__ne__', '__new__', '__next__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__setstate__', '__sizeof__', '__str__', '__subclasshook__']


In [12]:
print(type(iterador_metodo))
print(iterador_metodo.__next__())
print(next(iterador_metodo))

print(list(iterador_metodo))

<class 'list_iterator'>
1
2
[3, 4, 5, 6, 7, 8, 9, 0]


In [13]:
next(iterador_metodo)

StopIteration: 

In [14]:
del iterador_metodo
garbage_colector.collect(), garbage_colector.get_threshold()

(496, (700, 10, 10))

# <center> GENERADORES</center>
# <center> YIELD vs RETURN</center>

In [15]:
def lista_valores_return(limite):
    
    lista = []
    
    for valor in limite:
        lista.append(valor+1)
    
    return lista

valores = lista_valores_return(range(15))

for valor in valores:
    print(valor)
    
# elimina la variable creada, recupera el espacio de la variable liberandolo de la memoria con el garbage collector
del valores
garbage_colector.collect(), garbage_colector.get_threshold()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


(0, (700, 10, 10))

In [16]:
def lista_valores_generador(limite):
    
    for valor in limite:
        yield valor + 1

valores = lista_valores_generador(range(15))

for valor in valores:
    print(valor)
    
del valores
garbage_colector.collect(), garbage_colector.get_threshold()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15


(0, (700, 10, 10))

# <center>RECUPERANDO VALORES CON **NEXT** Y **LIST, SET**</center>
## <center> **NEXT** SIN CONTROL DE LIMITE DE VALORES DEL GENERADOR</center>

In [17]:
def lenguajes():
    yield 'Python'
    yield 'Java'
    yield 'Scala'
    yield 'C'
    yield 'C++'
    
valores = lenguajes()

print(next(valores))
print('estado de suspension, cualquier otra operacion, antes de que el generador retorne otro valor')
print(next(valores))
print('estado de suspension, cualquier otra operacion, anres de que el generador retorne otro valor')
print(next(valores))
print('estado de suspension, cualquier otra operacion, antes de que el generador retorne otro valor')
print(next(valores))
print('estado de suspension, cualquier otra operacion, antes de que el generador retorne otro valor')
print(next(valores))
print('estado de suspension, cualquier otra operacion, antes de que el generador retorne otro valor')
print(next(valores))

Python
estado de suspension, cualquier otra operacion, antes de que el generador retorne otro valor
Java
estado de suspension, cualquier otra operacion, anres de que el generador retorne otro valor
Scala
estado de suspension, cualquier otra operacion, antes de que el generador retorne otro valor
C
estado de suspension, cualquier otra operacion, antes de que el generador retorne otro valor
C++
estado de suspension, cualquier otra operacion, antes de que el generador retorne otro valor


StopIteration: 

In [18]:
del valores
garbage_colector.collect(), garbage_colector.get_threshold()

(489, (700, 10, 10))

## <center> **NEXT** CONTROLANDO EL LIMITE DE VALORES DEL GENERADOR</center>

In [19]:
def iterador_generadores(generador):

    while generador:        
        try:
            valor_recuperado = next(generador)
        except StopIteration:
            print('Finalizo iteracion')
            break
        else:
            print(valor_recuperado)

    
iterador_generadores(lenguajes())

Python
Java
Scala
C
C++
Finalizo iteracion


## <center> **SET y LIST**</center>

In [20]:
print(list(lenguajes()))
print(set(lenguajes())) # puede retornar orden distinto al especificado, elimina duplicados al generador.

['Python', 'Java', 'Scala', 'C', 'C++']
{'C', 'Python', 'Java', 'C++', 'Scala'}


# <center> GENERADOR USANDO **LIST COMPREHENSION**

In [21]:
lenguajes_generador = (valor for valor in ['Python', 'Java', 'Scala','C','C++'])
print(lenguajes_generador)

<generator object <genexpr> at 0x7f9190224190>


In [22]:
iterador_generadores(lenguajes_generador)

# elimina la variable creada, recupera el espacio de la variable liberandolo de la memoria con el garbage collector
del lenguajes_generador
garbage_colector.collect(), garbage_colector.get_threshold()

Python
Java
Scala
C
C++
Finalizo iteracion


(0, (700, 10, 10))

# <center> YIELD FROM</center>

In [23]:
def lista_generador():
    
    yield from 'JorgeCardona'
    
iterador_generadores(lista_generador())

J
o
r
g
e
C
a
r
d
o
n
a
Finalizo iteracion


# <center> **YIELD & YIELD FROM** DENTRO DE OTRO YIELD</center>

In [24]:
def astronomia():
    yield 'Sol'
    yield 'Luna'
    yield 'Estrellas'
    yield 'Galaxias'
    yield 'Planetas'   

def numeros():
    yield 'Uno'
    yield from lenguajes() # yield from, recupera todos los valores del otro generador
    yield 'Dos'
    yield lenguajes() # sino se usa yield from, retorna otro objeto Generador en vez de los valores.
    yield 'Tres'
    yield from astronomia() 
    yield 'Cuatro'
    yield lista_generador()
    yield 'Cinco'

iterador_generadores(numeros())

Uno
Python
Java
Scala
C
C++
Dos
<generator object lenguajes at 0x7f9190224200>
Tres
Sol
Luna
Estrellas
Galaxias
Planetas
Cuatro
<generator object lista_generador at 0x7f9190224200>
Cinco
Finalizo iteracion


# <center> METODOS DE UNA FUNCION, VARIABLE U OBJETO GENERADOR</center>

In [25]:
valores = lista_valores_generador(range(10))

print(dir(valores))

del valores
garbage_colector.collect(), garbage_colector.get_threshold()

['__class__', '__del__', '__delattr__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__iter__', '__le__', '__lt__', '__name__', '__ne__', '__new__', '__next__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', 'close', 'gi_code', 'gi_frame', 'gi_running', 'gi_yieldfrom', 'send', 'throw']


(0, (700, 10, 10))

# <center> **.\__\___next\__\___()**</center>

In [26]:
def iterador_de_lista_de_generadres(lista_funciones):    
    # itera la lista de funciones con generadores
    while lista_funciones:
        
        print(lista_funciones, len(lista_funciones))
        
        try:
            # obtiene el primer valor de la lista, que es una objeto o funcion generadora
            actual = lista_funciones.pop(0)
            # obtiene el primer valor del generador, del objeto generador de la lista anterior
            valor_recuoerado_del_generador = actual.__next__()                   
        except StopIteration:
            # para evaluar si el Generator Actual, ya no tiene mas elementos 
            # ya que no todos los generadores tienen el mismo tamano
            import types
            print(f"Finalizo iteracion del {actual} -> {' ya que no hay mas elementos' if isinstance(actual, types.GeneratorType) else 'No es una expresion Generadora'} , es una  {type(actual)}")
            #break
        else:
            # adiciona de nuevo el objeto generador a la lista, porque ya no existe en la lista
            # el objeto generador recuperado de la lista, ya no contiene el primer valor
            # pues este fue liberado y guardado en la variable valor_recuoerado_del_generador
            lista_funciones.append(actual)
            
            # es el valor recuperado del generador de la lista actual
            print(valor_recuoerado_del_generador) 
        
iterador_de_lista_de_generadres([lista_generador(), lenguajes(), astronomia(), numeros()])

[<generator object lista_generador at 0x7f9190224510>, <generator object lenguajes at 0x7f9190224580>, <generator object astronomia at 0x7f91902247b0>, <generator object numeros at 0x7f9190224820>] 4
J
[<generator object lenguajes at 0x7f9190224580>, <generator object astronomia at 0x7f91902247b0>, <generator object numeros at 0x7f9190224820>, <generator object lista_generador at 0x7f9190224510>] 4
Python
[<generator object astronomia at 0x7f91902247b0>, <generator object numeros at 0x7f9190224820>, <generator object lista_generador at 0x7f9190224510>, <generator object lenguajes at 0x7f9190224580>] 4
Sol
[<generator object numeros at 0x7f9190224820>, <generator object lista_generador at 0x7f9190224510>, <generator object lenguajes at 0x7f9190224580>, <generator object astronomia at 0x7f91902247b0>] 4
Uno
[<generator object lista_generador at 0x7f9190224510>, <generator object lenguajes at 0x7f9190224580>, <generator object astronomia at 0x7f91902247b0>, <generator object numeros at 0x

# <center> **ENVIAR VALORES A UN GENERADOR**</center>

In [27]:
def multiplicar_valor():
    while True:
        x,y = yield
        yield x + y
gen = multiplicar_valor()

In [28]:
for i in range(10,20):
    next(gen)
    x = i//2
    y = i
    
    resultado = gen.send([x,y])
    
    print(f'el valor de {x} + {y} es {resultado}')
    
del gen
garbage_colector.collect(), garbage_colector.get_threshold()

el valor de 5 + 10 es 15
el valor de 5 + 11 es 16
el valor de 6 + 12 es 18
el valor de 6 + 13 es 19
el valor de 7 + 14 es 21
el valor de 7 + 15 es 22
el valor de 8 + 16 es 24
el valor de 8 + 17 es 25
el valor de 9 + 18 es 27
el valor de 9 + 19 es 28


(0, (700, 10, 10))

# <center> **ENVIAR VALORES A UN GENERADOR CON CONDICIONALES**</center>

In [29]:
def multiplicar_valor_impar(generador):
    
    while generador:
        # obtiene el valor actual del iterador en el generador
        actual = next(generador)
        print(f'El valor del iterador es {actual}')
        
        # si el valor del generador no es par se le envian parametros
        if actual%2 !=-0:
            
            # parametros a capturar con el metodo send
            x,y = yield
            
            # define las variables a capturar
            if (x and y) is not None:
                yield f'el valor de ({x} x {actual}) + ({y} x {actual} x {2}) es ({x * actual} + {y * actual * 2}) = {x*actual + y*actual*2}'
                print()
        else:
            print(f'El valor {actual} -->, no es un numero impar')
            print()
                
# define el numero maximo de objetos generadores creados
iterador = (i for i in range(10))
gen = multiplicar_valor_impar(iterador)

In [30]:
# rango superior al total de objetos generadores
for i in range(99):
    
    try:
        next(gen)
    except Exception:
        print('Iteracion Finalizada')
        break
    else:
        resultado_parametros = gen.send([i+2,i+5])
        print(resultado_parametros)
        
del gen, iterador
garbage_colector.collect(), garbage_colector.get_threshold()

El valor del iterador es 0
El valor 0 -->, no es un numero impar

El valor del iterador es 1
el valor de (2 x 1) + (5 x 1 x 2) es (2 + 10) = 12

El valor del iterador es 2
El valor 2 -->, no es un numero impar

El valor del iterador es 3
el valor de (3 x 3) + (6 x 3 x 2) es (9 + 36) = 45

El valor del iterador es 4
El valor 4 -->, no es un numero impar

El valor del iterador es 5
el valor de (4 x 5) + (7 x 5 x 2) es (20 + 70) = 90

El valor del iterador es 6
El valor 6 -->, no es un numero impar

El valor del iterador es 7
el valor de (5 x 7) + (8 x 7 x 2) es (35 + 112) = 147

El valor del iterador es 8
El valor 8 -->, no es un numero impar

El valor del iterador es 9
el valor de (6 x 9) + (9 x 9 x 2) es (54 + 162) = 216

Iteracion Finalizada


(0, (700, 10, 10))

# <center> **EXCEPCIONES EN UN GENERADOR**</center>

In [31]:
numeros = (valor for valor in range(10))

def iterador_generadores(generador):

    while generador:        
        try:            
            valor = next(generador)
            print(valor)
            
            # criterio de parada
            if valor > 5:
                generador.throw(ValueError("EL VALOR HA SUPERADO EL LIMITE PERMITIDO"))
                
        except StopIteration:
            print('Finalizo iteracion')
            break
            
iterador_generadores(numeros)

0
1
2
3
4
5
6


ValueError: EL VALOR HA SUPERADO EL LIMITE PERMITIDO

# <center> **DETENER O FINALIZAR UN GENERADOR**</center>

In [32]:
numeros = (valor for valor in range(10))

def iterador_generadores(generador):

    while generador:        
        try:            
            valor = next(generador)               
        except StopIteration:
            print('Finalizo iteracion')
            break
        else:
            # criterio de parada
            if valor >= 5:
                generador.close()
            print(valor) 

iterador_generadores(numeros)

0
1
2
3
4
5
Finalizo iteracion


# <center> **PIPE O CANALIZACIONES CON GENERADORES**</center>

In [33]:
def fibonacci(nums):
    x, y = 0, 1
    for _ in range(nums):
        x, y = y, x+y
        yield x

def cuadrado(nums):
    for num in nums:
        yield num**2

print(list(fibonacci(10)))
print(list(cuadrado(fibonacci(10))))
print(sum(cuadrado(fibonacci(10))))
garbage_colector.collect(), garbage_colector.get_threshold()

[1, 1, 2, 3, 5, 8, 13, 21, 34, 55]
[1, 1, 4, 9, 25, 64, 169, 441, 1156, 3025]
4895


(1063, (700, 10, 10))

# <center> **CONSUMO DE MEMORIA EN BYTES DE UNA LISTA VS UN GENERADOR**</center>

In [34]:
import sys

lista = [i * 2 for i in range(10000)]
print(sys.getsizeof(lista))

del lista
garbage_colector.collect(), garbage_colector.get_threshold()

85176


(0, (700, 10, 10))

In [35]:
generador = (i ** 2 for i in range(10000))

while generador: 

    try:
        siguiente = next(generador)
    except StopIteration:
        print('Finalizo iteracion')
        break
    else:
        print(f'tamano del valor generado {sys.getsizeof(siguiente)}, tamano del generador {sys.getsizeof(generador)}, valor {siguiente}')
        
del generador
garbage_colector.collect(), garbage_colector.get_threshold()

tamano del valor generado 24, tamano del generador 104, valor 0
tamano del valor generado 28, tamano del generador 104, valor 1
tamano del valor generado 28, tamano del generador 104, valor 4
tamano del valor generado 28, tamano del generador 104, valor 9
tamano del valor generado 28, tamano del generador 104, valor 16
tamano del valor generado 28, tamano del generador 104, valor 25
tamano del valor generado 28, tamano del generador 104, valor 36
tamano del valor generado 28, tamano del generador 104, valor 49
tamano del valor generado 28, tamano del generador 104, valor 64
tamano del valor generado 28, tamano del generador 104, valor 81
tamano del valor generado 28, tamano del generador 104, valor 100
tamano del valor generado 28, tamano del generador 104, valor 121
tamano del valor generado 28, tamano del generador 104, valor 144
tamano del valor generado 28, tamano del generador 104, valor 169
tamano del valor generado 28, tamano del generador 104, valor 196
tamano del valor generad

(0, (700, 10, 10))

# <center> **VELOCIDAD DE PROCESAMIENTO VS CONSUMO DE MEMORIA DE UNA LISTA VS UN GENERADOR**</center>

In [36]:
import cProfile
LIMITE = 1_000_000

# comprehension list, 1 solo llamado para crear la lista
cProfile.run(f'sum([i * 2 for i in range({LIMITE})])')

         5 function calls in 0.213 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        1    0.194    0.194    0.194    0.194 <string>:1(<listcomp>)
        1    0.010    0.010    0.213    0.213 <string>:1(<module>)
        1    0.000    0.000    0.213    0.213 {built-in method builtins.exec}
        1    0.010    0.010    0.010    0.010 {built-in method builtins.sum}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




In [37]:
# comprehension list, 1 solo llamado para crear el set
cProfile.run(f'sum((i * 2 for i in range({LIMITE})))')

         1000005 function calls in 2.030 seconds

   Ordered by: standard name

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
  1000001    1.100    0.000    1.100    0.000 <string>:1(<genexpr>)
        1    0.000    0.000    2.030    2.030 <string>:1(<module>)
        1    0.000    0.000    2.030    2.030 {built-in method builtins.exec}
        1    0.930    0.930    2.030    2.030 {built-in method builtins.sum}
        1    0.000    0.000    0.000    0.000 {method 'disable' of '_lsprof.Profiler' objects}




# <center> TIPOS DE INSTANCIA</center>

# <center> SINCRONISMO - HILOS - PROCESOS - ASINCRONISMO</center>

<center>
    <img width="50%" lign="center" src="ejecuciones.png">
</center>

# <center> EJECUCION **SINCRONA** o **SERIAL**</center>

In [38]:
from datetime import datetime
from time import sleep
LIMITE = 11

def dormir_sincrono(tiempo):
    print('corriendo tarea de tiempo',tiempo)
    sleep(tiempo)
    return tiempo

In [39]:
%%time
datos = []

for tiempo_espera in range(1, LIMITE):
    datos.append(dormir_sincrono(tiempo_espera))

print(datos)
print(f'suma de valores de tiempo evaluados {sum(datos)}')

del datos
garbage_colector.collect(), garbage_colector.get_threshold()

corriendo tarea de tiempo 1
corriendo tarea de tiempo 2
corriendo tarea de tiempo 3
corriendo tarea de tiempo 4
corriendo tarea de tiempo 5
corriendo tarea de tiempo 6
corriendo tarea de tiempo 7
corriendo tarea de tiempo 8
corriendo tarea de tiempo 9
corriendo tarea de tiempo 10
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
suma de valores de tiempo evaluados 55
CPU times: user 124 ms, sys: 23 ms, total: 147 ms
Wall time: 55.1 s


(0, (700, 10, 10))

# <center> EJECUCION **ASINCRONA** Multitarea cooperativa</center>

In [40]:
import asyncio

# para .ipynb, event loop esta listo, usar await
# await main()

# para .py, usar asyncio.run
# asyncio.run(funcion_asincrona())

async def corrutina(tiempo:int) -> int:
    print('corriendo tarea de tiempo',tiempo)
    await asyncio.sleep(tiempo)
    return tiempo

datos = []
resul = []

inicia = datetime.now()

# crea el listado de corrutinas
for tiempo_espera in range(1, LIMITE):
    datos.append(asyncio.gather(corrutina(tiempo_espera)))
    
# recupera los valores de las corrutinas
for corrutina in datos:
    valor = await corrutina
    resul.append(valor[0])

finaliza = datetime.now()
print(f'Wall time: {finaliza.second - inicia.second} s') 
print(resul)

del datos, resul, inicia, finaliza
garbage_colector.collect(), garbage_colector.get_threshold()

corriendo tarea de tiempo 1
corriendo tarea de tiempo 2
corriendo tarea de tiempo 3
corriendo tarea de tiempo 4
corriendo tarea de tiempo 5
corriendo tarea de tiempo 6
corriendo tarea de tiempo 7
corriendo tarea de tiempo 8
corriendo tarea de tiempo 9
corriendo tarea de tiempo 10
Wall time: 10 s
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]


(0, (700, 10, 10))

# <center> EJECUCION **HILO** Subprocesamiento múltiple</center>
## <center> Este módulo construye interfaces de hilado de alto nivel sobre el módulo de más bajo nivel _thread. 

In [41]:
# libreria para procesamiento con hilos
from threading import Thread

# libreria que permite guardar los valores retornados en una cola
from queue import Queue as cola_classica

def almacenar_en_cola(f):
    def wrapper(*args):
        almacenar_resultados.put(f(*args))
    return wrapper

@almacenar_en_cola
def dormir_sincrono(tiempo):
    print('corriendo tarea de tiempo',tiempo)
    sleep(tiempo)
    return tiempo

def obtener_resultado_cola_tarea(cola):
    
    valores = []
    for indice_tarea in range(cola.qsize()):
        valores.append(cola.get(indice_tarea))
    
    return valores

In [42]:
%%time

# instancia de la cola para adicionar los resultados
almacenar_resultados = cola_classica()

# almacena las instancias generadas
listado_instancias = []   
    
# genera la instancias necesarias para ejecutar, las inicializa y las guarda en la lista
for tiempo_espera in range(1, LIMITE):
  
    instancia = Thread(name=f'ejecucion de tarea # {tiempo_espera}', target=dormir_sincrono, args=(tiempo_espera,))

    instancia.start() # Comienza la actividad del hilo. Esto debe llamarse como máximo una vez por objeto de hilo.
    
    listado_instancias.append(instancia)

# recorre la instancias generadass en la lista de instancias y bloque las ejecuciones hasta que termine    
for tipo_instancia in listado_instancias:    
        tipo_instancia.join() # Espera a que salgan los hilos de trabajo. Espera a que el hilo termine. Esto bloquea el hilo llamador hasta que el hilo cuyo método join() es llamado finalice.

# ejecuta la funcion que se encarga de recuperar los resultados de las ejecuciones        
obtener_resultado_cola_tarea(almacenar_resultados)

# elimina las variables creaadas para la ejecucion
del almacenar_resultados, listado_instancias

# muestra la cantidad de objetos que se mantienen despues de la ejecucion  y el espacio ocupado por estos
garbage_colector.collect(), garbage_colector.get_threshold()

corriendo tarea de tiempo 1
corriendo tarea de tiempo 2
corriendo tarea de tiempo 3
corriendo tarea de tiempo 4
corriendo tarea de tiempo 5
corriendo tarea de tiempo 6
corriendo tarea de tiempo 7
corriendo tarea de tiempo 8
corriendo tarea de tiempo 9
corriendo tarea de tiempo 10
CPU times: user 78.5 ms, sys: 0 ns, total: 78.5 ms
Wall time: 10.1 s


(0, (700, 10, 10))

## <center> INSTANCIAS CON **LIST COMPREHENSION**

In [43]:
%%time

# instancia de la cola para adicionar los resultados
almacenar_resultados = cola_classica()

# almacena las instancias generadas
listado_instancias = []   

# genera la instancias necesarias para ejecutar, las inicializa y las guarda en la lista
instancias =  [Thread(name=f'ejecucion de tarea # {tiempo_espera}', target=dormir_sincrono, args=(tiempo_espera,)) for tiempo_espera in range(1, LIMITE)]
    
# recorre cada instancia, la inicializa y luego la guarda en la lista de instancias
for instancia in instancias:

    instancia.start() # Comienza la actividad del hilo. Esto debe llamarse como máximo una vez por objeto de hilo.
    
    listado_instancias.append(instancia)

# recorre la instancias generadass en la lista de instancias y bloque las ejecuciones hasta que termine    
for tipo_instancia in listado_instancias:    
        tipo_instancia.join() # Espera a que salgan los hilos de trabajo. Espera a que el hilo termine. Esto bloquea el hilo llamador hasta que el hilo cuyo método join() es llamado finalice.

# ejecuta la funcion que se encarga de recuperar los resultados de las ejecuciones        
obtener_resultado_cola_tarea(almacenar_resultados)

# elimina las variables creaadas para la ejecucion
del almacenar_resultados, listado_instancias

# muestra la cantidad de objetos que se mantienen despues de la ejecucion  y el espacio ocupado por estos
garbage_colector.collect(), garbage_colector.get_threshold()

corriendo tarea de tiempo 1
corriendo tarea de tiempo 2
corriendo tarea de tiempo 3
corriendo tarea de tiempo 4
corriendo tarea de tiempo 5
corriendo tarea de tiempo 6
corriendo tarea de tiempo 7
corriendo tarea de tiempo 8
corriendo tarea de tiempo 9
corriendo tarea de tiempo 10
CPU times: user 42.4 ms, sys: 0 ns, total: 42.4 ms
Wall time: 10 s


(0, (700, 10, 10))

# <center> EJECUCION **PROCESO MULTIPLE**</center>
## <center> Este módulo permite crear procesos (spawning) utilizando una API similar al módulo threading. ofrece concurrencia tanto local como remota. permite aprovechar al máximo múltiples procesadores en una máquina determinada. 

In [44]:
# libreria que permite usar la computacion paralela
from multiprocessing import Process

# libreria que permite guardar los valores retornados en una cola
from multiprocessing import Queue as cola_proceso

def almacenar_en_cola_proceso(f):
    def wrapper(*args):
        almacenar_resultados_proceso.put(f(*args))
    return wrapper

@almacenar_en_cola_proceso
def dormir_sincrono(tiempo):
    print('corriendo tarea de tiempo',tiempo)
    sleep(tiempo)
    return tiempo

def obtener_resultado_cola_tarea_while(cola):
    
    valores = []
    while not cola.empty():
        result = cola.get()
        valores.append(result)    
    return valores

In [45]:
%%time

# instancia de la cola para adicionar los resultados
almacenar_resultados_proceso = cola_proceso()

# almacena las instancias generadas
listado_instancias = []

for tiempo_espera in range(1, LIMITE):
  
    instancia = Process(name=f'ejecucion de tarea # {tiempo_espera}', target=dormir_sincrono, args=(tiempo_espera,))
    instancia.start() # Comienza la actividad del proceso. Esto debe llamarse como máximo una vez por objeto de proceso.  
    listado_instancias.append(instancia)

for tipo_instancia in listado_instancias:        
        tipo_instancia.join() # Espera a que salgan los procesos de trabajo. Se debe llamar close() o terminate() antes de usar join().

obtener_resultado_cola_tarea_while(almacenar_resultados_proceso)

del almacenar_resultados_proceso, listado_instancias
garbage_colector.collect(), garbage_colector.get_threshold()

corriendo tarea de tiempo corriendo tarea de tiempo1 
corriendo tarea de tiempocorriendo tarea de tiempo 2 
3corriendo tarea de tiempo4
5
 corriendo tarea de tiempocorriendo tarea de tiempo 
 
76corriendo tarea de tiempocorriendo tarea de tiempocorriendo tarea de tiempo 
8
  109

CPU times: user 84.8 ms, sys: 96.1 ms, total: 181 ms
Wall time: 10.2 s


(0, (700, 10, 10))

# <center> TOTAL PROCESADORES

In [46]:
import psutil
total_cpu = psutil.cpu_count()
mitad_procesamiento = total_cpu//2
total_cpu, mitad_procesamiento

(4, 2)

# <center> EJECUCION **THREAD POOL DE MULTIPROCESO**</center>

In [47]:
def dormir_sincrono_multiprocessing(tiempo):
    print('corriendo tarea de tiempo ',tiempo)
    sleep(tiempo)
    return tiempo

# <center> **PROCESADORES POR DEFECTO**</center>

In [48]:
# libreria quenpermite usar multihilos con la libreria multiprocessing
from multiprocessing.pool import ThreadPool

In [49]:
%%time

pool = ThreadPool()
multihilo = pool.map(dormir_sincrono_multiprocessing, range(1, LIMITE))
pool.close()
pool.join()

print(multihilo)

del pool, multihilo
garbage_colector.collect(), garbage_colector.get_threshold()

corriendo tarea de tiempo corriendo tarea de tiempo  2
 1
corriendo tarea de tiempo  3
corriendo tarea de tiempo  4
corriendo tarea de tiempo  5
corriendo tarea de tiempo  6
corriendo tarea de tiempo  7
corriendo tarea de tiempo  8
corriendo tarea de tiempo  9
corriendo tarea de tiempo  10
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
CPU times: user 43.3 ms, sys: 2.92 ms, total: 46.3 ms
Wall time: 18.1 s


(0, (700, 10, 10))

# <center> **PROCESADORES ASIGNADOS** processes</center>

In [50]:
%%time

with ThreadPool(processes=mitad_procesamiento) as pool_de_hilos:
    multihilo = pool_de_hilos.map(dormir_sincrono_multiprocessing, range(1, LIMITE))

print(multihilo)

del pool_de_hilos, multihilo
garbage_colector.collect(), garbage_colector.get_threshold()

corriendo tarea de tiempo  1
corriendo tarea de tiempo  3
corriendo tarea de tiempo  2
corriendo tarea de tiempo  4
corriendo tarea de tiempo  5
corriendo tarea de tiempo  7
corriendo tarea de tiempo  6
corriendo tarea de tiempo  8
corriendo tarea de tiempo  9
corriendo tarea de tiempo  10
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
CPU times: user 40.4 ms, sys: 10.1 ms, total: 50.4 ms
Wall time: 33.1 s


(0, (700, 10, 10))

# <center> EJECUCION **POOL DE MULTIPROCESO**</center>

# <center> **PROCESADORES POR DEFECTO**</center>

In [51]:
# libreria quenpermite usar multiprocesos con la libreria multiprocessing
from multiprocessing.pool import Pool

In [52]:
%%time

pool = Pool()
multiproceso = pool.map(dormir_sincrono_multiprocessing, range(1, LIMITE))
pool.close() # Impide que se envíen más tareas a la piscina (pool). Una vez que se hayan completado todas las tareas, se cerrarán los procesos de trabajo.
pool.join() # Espera a que salgan los procesos de trabajo. Se debe llamar close() o terminate() antes de usar join().

print(multiproceso)

del pool, multiproceso
garbage_colector.collect(), garbage_colector.get_threshold()

corriendo tarea de tiempo corriendo tarea de tiempo corriendo tarea de tiempo corriendo tarea de tiempo    4 1
32


corriendo tarea de tiempo  5
corriendo tarea de tiempo  6
corriendo tarea de tiempo  7
corriendo tarea de tiempo  8
corriendo tarea de tiempo  9
corriendo tarea de tiempo  10
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
CPU times: user 77.3 ms, sys: 27.8 ms, total: 105 ms
Wall time: 18.1 s


(0, (700, 10, 10))

## <center> **PROCESOS ADMINISTRADOR DE CONTEXTO**</center>

In [53]:
%%time
with Pool() as pool_de_procesos:
    multiproceso = pool_de_procesos.map(dormir_sincrono_multiprocessing, range(1, LIMITE))

print(multiproceso)

del pool_de_procesos, multiproceso
garbage_colector.collect(), garbage_colector.get_threshold()

corriendo tarea de tiempo corriendo tarea de tiempo corriendo tarea de tiempo corriendo tarea de tiempo     3124



corriendo tarea de tiempo  5
corriendo tarea de tiempo  6
corriendo tarea de tiempo  7
corriendo tarea de tiempo  8
corriendo tarea de tiempo  9
corriendo tarea de tiempo  10
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
CPU times: user 73.5 ms, sys: 34.7 ms, total: 108 ms
Wall time: 18.1 s


(0, (700, 10, 10))

# <center> **PROCESADORES ASIGNADOS** processes</center>

In [54]:
%%time
with Pool(processes=mitad_procesamiento) as pool_de_procesos:
    multiproceso = pool_de_procesos.map(dormir_sincrono_multiprocessing, range(1, LIMITE))
    
print(multiproceso)

del pool_de_procesos, multiproceso
garbage_colector.collect(), garbage_colector.get_threshold()

corriendo tarea de tiempo corriendo tarea de tiempo  1
 3
corriendo tarea de tiempo  2
corriendo tarea de tiempo  5
corriendo tarea de tiempo  4
corriendo tarea de tiempo  7
corriendo tarea de tiempo  6
corriendo tarea de tiempo  9
corriendo tarea de tiempo  8
corriendo tarea de tiempo  10
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
CPU times: user 87.3 ms, sys: 29.8 ms, total: 117 ms
Wall time: 33.1 s


(0, (700, 10, 10))

# <center> EJECUCION **THREAD POOL EXECUTOR DE CONCURRENT.FUTURES**</center>
## <center> Este módulo provee una interfaz de alto nivel para ejecutar invocables de forma **asincrónica**. Usando **ThreadPoolExecutor, o ProcessPoolExecutor**. Ambos implementan la misma interfaz definida por la **clase abstracta Executor** compatible con asyncio.

In [55]:
def dormir_concurrent_futures(tiempo):
    print('corriendo tarea de tiempo ',tiempo)
    sleep(tiempo)
    return tiempo

# <center> **PROCESADORES POR DEFECTO**</center>

In [56]:
# importa librerias para trabajar concurrencia
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import as_completed

In [57]:
%%time

executor = ThreadPoolExecutor()

futures = []
resultado = []
for tiempo in range(1, LIMITE):
    future = executor.submit(dormir_concurrent_futures, tiempo)
    futures.append(future)

for future in as_completed(futures):
    resultado.append(future.result())

executor.shutdown()
print(resultado)

del executor, futures, resultado
garbage_colector.collect(), garbage_colector.get_threshold()

corriendo tarea de tiempo  1
corriendo tarea de tiempo  2
corriendo tarea de tiempo  3
corriendo tarea de tiempo  4
corriendo tarea de tiempo  5
corriendo tarea de tiempo  6
corriendo tarea de tiempo  7
corriendo tarea de tiempo  8
corriendo tarea de tiempo  9
corriendo tarea de tiempo  10
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
CPU times: user 40.7 ms, sys: 10.1 ms, total: 50.9 ms
Wall time: 12 s


(0, (700, 10, 10))

# <center> **EL ADMINISTRADOR DE CONTEXTO** NO GARANTIZA EL ORDEN</center>

In [58]:
%%time

futures = []
resultado = []

with ThreadPoolExecutor() as executor:
    for tiempo in range(1, LIMITE):
        future = executor.submit(dormir_concurrent_futures, tiempo)
        futures.append(future)    

    for future in as_completed(futures):
        resultado.append(future.result())
    
print(resultado)

del executor, futures, resultado
garbage_colector.collect(), garbage_colector.get_threshold()

corriendo tarea de tiempo  1
corriendo tarea de tiempo  2
corriendo tarea de tiempo  3
corriendo tarea de tiempo  4
corriendo tarea de tiempo  5
corriendo tarea de tiempo  6
corriendo tarea de tiempo  7
corriendo tarea de tiempo  8
corriendo tarea de tiempo  9
corriendo tarea de tiempo  10
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
CPU times: user 37.7 ms, sys: 7.92 ms, total: 45.6 ms
Wall time: 12 s


(0, (700, 10, 10))

# <center> **PROCESADORES ASIGNADOS ADMINISTRADOR DE CONTEXTO** max_workers</center>

In [59]:
%%time

futures = []
resultado = []

with ThreadPoolExecutor(max_workers=mitad_procesamiento) as executor:
    for tiempo in range(1, LIMITE):
        future = executor.submit(dormir_concurrent_futures, tiempo)
        futures.append(future)    

    for future in as_completed(futures):
        resultado.append(future.result())
    
print(resultado)

del executor, futures, resultado
garbage_colector.collect(), garbage_colector.get_threshold()

corriendo tarea de tiempo  1
corriendo tarea de tiempo  2
corriendo tarea de tiempo  3
corriendo tarea de tiempo  4
corriendo tarea de tiempo  5
corriendo tarea de tiempo  6
corriendo tarea de tiempo  7
corriendo tarea de tiempo  8
corriendo tarea de tiempo  9
corriendo tarea de tiempo  10
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
CPU times: user 51 ms, sys: 0 ns, total: 51 ms
Wall time: 30.1 s


(0, (700, 10, 10))

# <center> **PROCESADORES ASIGNADOS MAP** MANTIENE EL ORDEN DE LA INFORMACION</center>

In [60]:
%%time

with ThreadPoolExecutor(max_workers=mitad_procesamiento) as executor:
    futures = executor.map(dormir_concurrent_futures, range(1, LIMITE))
list(futures) 

del executor, futures
garbage_colector.collect(), garbage_colector.get_threshold()

corriendo tarea de tiempo  1
corriendo tarea de tiempo  2
corriendo tarea de tiempo  3
corriendo tarea de tiempo  4
corriendo tarea de tiempo  5
corriendo tarea de tiempo  6
corriendo tarea de tiempo  7
corriendo tarea de tiempo  8
corriendo tarea de tiempo  9
corriendo tarea de tiempo  10
CPU times: user 48.9 ms, sys: 0 ns, total: 48.9 ms
Wall time: 30.1 s


(0, (700, 10, 10))

# <center> EJECUCION **PROCESS POOL EXECUTOR DE CONCURRENT.FUTURES**</center>

In [61]:
# importa librerias para trabajar concurrencia
from concurrent.futures import ProcessPoolExecutor

# <center> **PROCESADORES POR DEFECTO**</center>

In [62]:
%%time

executor = ProcessPoolExecutor()

futures = []
resultado = []
for tiempo in range(1, LIMITE):
    future = executor.submit(dormir_concurrent_futures, tiempo) # submit -> Programa el invocable, fn, para que se ejecute como fn(*args, **kwargs) y devuelve un objeto Future que representa la ejecución del invocable.
    futures.append(future)

# as_completed -> espera que se complete cada llamado de future para poder mostrar su resultado.
# usar as_completed -> cuando se una el metodo submit para correr una funcion
for future in as_completed(futures):
    resultado.append(future.result()) # .result() obtiene el resultado de ejecutor

executor.shutdown() # Indica al ejecutor que debe liberar todos los recursos que está utilizando cuando los futuros actualmente pendientes de ejecución finalicen.

print(resultado)

del executor, futures, resultado
garbage_colector.collect(), garbage_colector.get_threshold()

corriendo tarea de tiempo  1
corriendo tarea de tiempo  2corriendo tarea de tiempo 
corriendo tarea de tiempo  4
 3
corriendo tarea de tiempo  5
corriendo tarea de tiempo  6
corriendo tarea de tiempo  7
corriendo tarea de tiempo  8
corriendo tarea de tiempo  9
corriendo tarea de tiempo  10
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
CPU times: user 71.1 ms, sys: 74.3 ms, total: 145 ms
Wall time: 18.1 s


(0, (700, 10, 10))

# <center> **EL ADMINISTRADOR DE CONTEXTO** NO GARANTIZA EL ORDEN</center>

In [63]:
%%time

futures = []
resultado = []

with ProcessPoolExecutor() as executor:
    for tiempo in range(1, LIMITE):
        future = executor.submit(dormir_concurrent_futures, tiempo)
        futures.append(future)    

    for future in as_completed(futures):
        resultado.append(future.result())
    
print(resultado)

del executor, futures, resultado
garbage_colector.collect(), garbage_colector.get_threshold()

corriendo tarea de tiempo  1corriendo tarea de tiempo  2

corriendo tarea de tiempo  corriendo tarea de tiempo  4
3
corriendo tarea de tiempo  5
corriendo tarea de tiempo  6
corriendo tarea de tiempo  7
corriendo tarea de tiempo  8
corriendo tarea de tiempo  9
corriendo tarea de tiempo  10
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
CPU times: user 84.4 ms, sys: 45.6 ms, total: 130 ms
Wall time: 18.1 s


(0, (700, 10, 10))

# <center> **PROCESADORES POR DEFECTO USANDO MAP**</center>

In [64]:
%%time

with ProcessPoolExecutor() as executor:
    futures = executor.map(dormir_concurrent_futures, range(1, LIMITE))
list(futures)

del executor, futures
garbage_colector.collect(), garbage_colector.get_threshold()

corriendo tarea de tiempo  1corriendo tarea de tiempo 
2 
corriendo tarea de tiempo  corriendo tarea de tiempo  3
4
corriendo tarea de tiempo  5
corriendo tarea de tiempo  6
corriendo tarea de tiempo  7
corriendo tarea de tiempo  8
corriendo tarea de tiempo  9
corriendo tarea de tiempo  10
CPU times: user 71.8 ms, sys: 55 ms, total: 127 ms
Wall time: 18.1 s


(0, (700, 10, 10))

# <center> **PROCESADORES ASIGNADOS** max_workers</center>

In [65]:
%%time

futures = []
resultado = []
with ProcessPoolExecutor(max_workers=2) as executor:    
    futures = executor.map(dormir_concurrent_futures, range(1, LIMITE))
list(futures)  

del executor, futures, resultado
garbage_colector.collect(), garbage_colector.get_threshold()

corriendo tarea de tiempo corriendo tarea de tiempo  1 
2
corriendo tarea de tiempo  3
corriendo tarea de tiempo  4
corriendo tarea de tiempo  5
corriendo tarea de tiempo  6
corriendo tarea de tiempo  7
corriendo tarea de tiempo  8
corriendo tarea de tiempo  9
corriendo tarea de tiempo  10
CPU times: user 95.3 ms, sys: 10.3 ms, total: 106 ms
Wall time: 30.1 s


(0, (700, 10, 10))

# *****************************************************************************************************************************************************************************************************************************

# <center> HILOS I/O VS PROCESOS I/O</center>

In [66]:
cantidad = 100_000

def lectura_io(value):
    with open('demofile.txt', 'rb') as f:
        return f.read(value * 100)

## <center> HILOS I/O</center>

### <center> LIMITAR LA CANTIDAD DE TAREAS PARA NO DESBORDAR LA MEMORIA POR ALMACENAMIENTO DE RESULTADOS EN LA COLA</center>

In [67]:
%%time
almacenar_resultados = cola_classica()
listado_instancias = [] 

@almacenar_en_cola
def lectura_io_hilo(value):
    with open('demofile.txt', 'rb') as f:
        return f.read(value * 100)
    
for valor_lectura in range(1, cantidad//2):
  
    instancia = Thread(name=f'ejecucion de tarea # {valor_lectura}', target=lectura_io_hilo, args=(valor_lectura,))
    instancia.start()    
    listado_instancias.append(instancia)

    
for tipo_instancia in listado_instancias:    
        tipo_instancia.join()
        
resultado = obtener_resultado_cola_tarea(almacenar_resultados)
print(resultado[:100])

del almacenar_resultados, listado_instancias, resultado
garbage_colector.collect(), garbage_colector.get_threshold()

[b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge

(0, (700, 10, 10))

### <center> CAPACIDAD DE PROCESAR TODAS LAS TAREAS SIN DESBORDAR LA MEMORIA</center>

In [68]:
%%time

with ThreadPool() as multi_hilos:        
    resultado = multi_hilos.map(lectura_io, range(cantidad))
    print(len(resultado))
    
print(list(resultado)[:100])

del multi_hilos, resultado
garbage_colector.collect(), garbage_colector.get_threshold()

100000
[b'', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronis

(0, (700, 10, 10))

In [69]:
%%time

with ThreadPoolExecutor() as multi_hilos:        
    resultado = multi_hilos.map(lectura_io, range(cantidad))
    print(len(list(resultado)))
    
print(list(resultado))

del multi_hilos, resultado
garbage_colector.collect(), garbage_colector.get_threshold()

100000
[]
CPU times: user 20.7 s, sys: 54.2 s, total: 1min 14s
Wall time: 32.8 s


(0, (700, 10, 10))

## <center> PROCESOS I/O</center>

In [70]:
%%time
almacenar_resultados_proceso = cola_proceso()
listado_instancias = [] 

@almacenar_en_cola_proceso
def lectura_io_proceso(value):
    with open('demofile.txt', 'rb') as f:
        return f.read(value * 100)
    
for valor_lectura in range(1, cantidad//200):
  
    instancia = Process(name=f'ejecucion de tarea # {valor_lectura}', target=lectura_io_proceso, args=(valor_lectura,))
    instancia.start()    
    listado_instancias.append(instancia)

    
for tipo_instancia in listado_instancias:    
        tipo_instancia.join()
        
resultado = obtener_resultado_cola_tarea_while(almacenar_resultados_proceso)
print(resultado[:100])

del almacenar_resultados_proceso, listado_instancias, resultado
garbage_colector.collect(), garbage_colector.get_threshold()

[b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge

(0, (700, 10, 10))

In [71]:
%%time
with Pool() as multiprocessing:
    resultado = multiprocessing.map(lectura_io, range(cantidad))
    print(len(list(resultado)))
    
print(list(resultado)[:100])

del multiprocessing, resultado
garbage_colector.collect(), garbage_colector.get_threshold()

100000
[b'', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronis

(0, (700, 10, 10))

In [72]:
%%time
with ProcessPoolExecutor() as concurrent_futures:
    resultado = concurrent_futures.map(lectura_io, range(cantidad))
    
print(list(resultado)[:100])

del concurrent_futures, resultado
garbage_colector.collect(), garbage_colector.get_threshold()

[b'', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'jorge cardona, hilos, procesos, asincronismo, sincronismo', b'

(0, (700, 10, 10))

# <center> HILOS CPU VS PROCESOS CPU</center>

In [73]:
import math

inicio = 1_099_726_899_294_999
fin    = 1_099_726_899_299_999

PRIMES = range(inicio, fin) 

def is_prime(n):
    if n < 2 or n % 2 == 0:
        return False
    if n == 2:
        return True
    
    sqrt_n = int(math.floor(math.sqrt(n)))
    
    for i in range(3, sqrt_n + 1, 2):
        if n % i == 0:
            return False
    return True

print(f'total iteraciones {fin - inicio}')

total iteraciones 5000


## <center> SINCRONO CPU</center>

In [74]:
%%time

print(list(map(is_prime, PRIMES)))

[False, False, False, False, False, False, False, False, True, False, True, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False

## <center> HILOS CPU</center>

In [75]:
%%time
almacenar_resultados = cola_classica()
listado_instancias = [] 

@almacenar_en_cola
def is_prime_hilo(n):
    if n < 2 or n % 2 == 0:
        return False
    if n == 2:
        return True
    
    sqrt_n = int(math.floor(math.sqrt(n)))
    
    for i in range(3, sqrt_n + 1, 2):
        if n % i == 0:
            return False
    return True
    
for valor_lectura in PRIMES:
  
    instancia = Thread(name=f'ejecucion de tarea # {valor_lectura}', target=is_prime_hilo, args=(valor_lectura,))
    instancia.start()    
    listado_instancias.append(instancia)

    
for tipo_instancia in listado_instancias:    
        tipo_instancia.join()
        
resultado = obtener_resultado_cola_tarea(almacenar_resultados)
print(resultado[:100])

del almacenar_resultados, listado_instancias, resultado
garbage_colector.collect(), garbage_colector.get_threshold()

[False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False]
CPU times: user 5min 21s, sys: 0 ns, total: 5min 21s
Wall time: 5min 7s


(0, (700, 10, 10))

In [76]:
%%time
resultado = []

with ThreadPool() as executor:
    for number, prime in zip(PRIMES, executor.map(is_prime, PRIMES)):       
        resultado.append((number, prime))

print('cantidad de valores procesados', len(resultado))

del executor, resultado
garbage_colector.collect(), garbage_colector.get_threshold()

cantidad de valores procesados 5000
CPU times: user 5min 14s, sys: 0 ns, total: 5min 14s
Wall time: 5min 7s


(0, (700, 10, 10))

In [77]:
%%time
resultado = []

with ThreadPoolExecutor() as executor:
    for number, prime in zip(PRIMES, executor.map(is_prime, PRIMES)):       
        resultado.append((number, prime))

print('cantidad de valores procesados', len(resultado))
        
del executor, resultado
garbage_colector.collect(), garbage_colector.get_threshold()

cantidad de valores procesados 5000
CPU times: user 5min 39s, sys: 0 ns, total: 5min 39s
Wall time: 5min 22s


(0, (700, 10, 10))

## <center> PROCESOS CPU</center>

In [78]:
%%time
almacenar_resultados_proceso = cola_proceso()
listado_instancias = [] 

@almacenar_en_cola_proceso
def is_prime_proceso(n):
    if n < 2 or n % 2 == 0:
        return False
    if n == 2:
        return True
    
    sqrt_n = int(math.floor(math.sqrt(n)))
    
    for i in range(3, sqrt_n + 1, 2):
        if n % i == 0:
            return False
    return True
    
for valor_lectura in PRIMES:
  
    instancia = Process(name=f'ejecucion de tarea # {valor_lectura}', target=is_prime_proceso, args=(valor_lectura,))
    instancia.start()    
    listado_instancias.append(instancia)

    
for tipo_instancia in listado_instancias:    
        tipo_instancia.join()

print('cantidad de valores procesados', len(obtener_resultado_cola_tarea_while(almacenar_resultados_proceso)))

del almacenar_resultados_proceso, listado_instancias
garbage_colector.collect(), garbage_colector.get_threshold()

cantidad de valores procesados 5000
CPU times: user 35.9 s, sys: 6.65 s, total: 42.6 s
Wall time: 3min 23s


(0, (700, 10, 10))

In [79]:
%%time

resultado = []
with Pool() as executor:
    for number, prime in zip(PRIMES, executor.map(is_prime, PRIMES)):       
        resultado.append((number, prime))

print('cantidad de valores procesados', len(resultado))

del executor, resultado
garbage_colector.collect(), garbage_colector.get_threshold()

cantidad de valores procesados 5000
CPU times: user 203 ms, sys: 35.6 ms, total: 239 ms
Wall time: 3min 12s


(0, (700, 10, 10))

In [80]:
%%time

resultado = []
with ProcessPoolExecutor() as executor:
    for number, prime in zip(PRIMES, executor.map(is_prime, PRIMES)):       
        resultado.append((number, prime))

print('cantidad de valores procesados', len(resultado))
        
del executor, resultado
garbage_colector.collect(), garbage_colector.get_threshold()

cantidad de valores procesados 5000
CPU times: user 2.42 s, sys: 491 ms, total: 2.91 s
Wall time: 4min 38s


(0, (700, 10, 10))

## <center> NUMBA CPU</center>

In [81]:
#pip install numba # Numba es el paquete compilador, esto depende de llvmlite.
#pip install llvmlite # llvmlite es un paquete de enlace ligero para las API de LLVM, depende de LLVM, LLVM es el marco del compilador JIT para producir código ejecutable a partir de varias entradas.
!pip show numba

Name: numba
Version: 0.55.1
Summary: compiling Python code using LLVM
Home-page: https://numba.pydata.org
Author: 
Author-email: 
License: BSD
Location: /usr/local/lib/python3.10/site-packages
Requires: llvmlite, numpy, setuptools
Required-by: 


In [82]:
from numba import jit

@jit(nopython=True)
def is_prime_numba(n):
    if n < 2 or n % 2 == 0:
        return False
    if n == 2:
        return True
    
    sqrt_n = int(math.floor(math.sqrt(n)))
    for i in range(3, sqrt_n + 1, 2):
        if n % i == 0:
            return False
    return True

In [83]:
%%time
print(f'total iteraciones {fin - inicio}')
print(list(map(is_prime_numba, PRIMES)))

garbage_colector.collect(), garbage_colector.get_threshold()

total iteraciones 5000
[False, False, False, False, False, False, False, False, True, False, True, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, Tr

(3644, (700, 10, 10))

In [84]:
%%time

inicio = 1_099_726_899_249_999
fin    = 1_099_726_899_299_999

PRIMES = range(inicio, fin) 

print(f'total iteraciones {fin - inicio}')
print(list(map(is_prime_numba, PRIMES)))
garbage_colector.collect(), garbage_colector.get_threshold()

total iteraciones 50000
[False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, True, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, False, Fals

(0, (700, 10, 10))

# *****************************************************************************************************************************************************************************************************************************

# <center> EJECUCION 1800 TAREAS DE 2 SEGUNDOS, TOMARIA 1 HORA EN EJECUCION SINCRONA</center>

In [85]:
import random
limite_tareas = 1800

## <center> ASINCRONO</center>

In [86]:
async def corrutina(tiempo:int) -> int:    
    await asyncio.sleep(tiempo)
    return random.randint(0, limite_tareas)

inicia = datetime.now()
lista = [corrutina(2) for i in range(limite_tareas)]

resultado = await asyncio.gather(*lista)

finaliza = datetime.now()
print(len(resultado))
print(f'Wall time: {finaliza.second - inicia.second} s')
print(resultado)

del lista, resultado, inicia, finaliza
garbage_colector.collect(), garbage_colector.get_threshold()

1800
Wall time: 2 s
[436, 525, 1005, 348, 1338, 624, 688, 761, 1792, 13, 268, 1511, 404, 972, 49, 1025, 677, 787, 970, 932, 1218, 1300, 1335, 1596, 902, 217, 1634, 1046, 338, 1271, 1572, 1788, 1583, 1328, 939, 440, 1789, 1608, 587, 1399, 170, 249, 309, 282, 287, 756, 833, 1162, 359, 152, 1076, 196, 1472, 481, 947, 657, 1761, 136, 643, 1662, 1015, 548, 81, 1248, 827, 451, 1622, 1459, 255, 397, 1796, 67, 920, 533, 209, 80, 1054, 206, 676, 748, 104, 927, 2, 502, 1065, 1547, 237, 1528, 1425, 756, 543, 117, 1168, 792, 1078, 829, 409, 1640, 1275, 1690, 148, 56, 1003, 1552, 114, 1217, 27, 1508, 970, 1404, 664, 662, 897, 816, 152, 1714, 1296, 1215, 1155, 435, 1209, 688, 149, 412, 644, 91, 591, 1628, 1368, 886, 604, 1709, 1596, 1197, 1148, 1369, 219, 1576, 511, 735, 236, 1300, 1111, 1394, 1781, 1146, 571, 124, 1439, 1068, 273, 150, 856, 507, 658, 1693, 39, 1295, 1110, 426, 1579, 882, 1435, 76, 836, 1335, 59, 1733, 1144, 580, 1121, 488, 1513, 688, 553, 1767, 474, 488, 1166, 1173, 1479, 1507, 477

(0, (700, 10, 10))

## <center> THREAD</center>

In [87]:
%%time

listado_instancias = []
# instancia de la cola para adicionar los resultados
almacenar_resultados = cola_classica()


@almacenar_en_cola
def dormir_sincrono(tiempo:int) -> int:    
    sleep(tiempo)
    return random.randint(0,limite_tareas)

    
for tiempo_espera in range(limite_tareas):
  
    instancia = Thread(name=f'ejecucion de tarea # {tiempo_espera}', target=dormir_sincrono, args=(2,))

    instancia.start()
    
    listado_instancias.append(instancia)

    
for tipo_instancia in listado_instancias:    
        tipo_instancia.join()

resultado = obtener_resultado_cola_tarea(almacenar_resultados)
print(len(resultado))
print(resultado)

del listado_instancias, almacenar_resultados, instancia, resultado
garbage_colector.collect(), garbage_colector.get_threshold()

1800
[48, 1690, 222, 1041, 167, 1234, 445, 1642, 819, 1504, 287, 1151, 376, 722, 443, 1, 974, 986, 661, 1593, 385, 1536, 471, 510, 156, 705, 811, 1256, 99, 1735, 769, 1351, 1198, 43, 147, 1199, 1241, 1314, 510, 957, 417, 261, 1573, 829, 1371, 840, 917, 1625, 1777, 560, 1755, 1676, 1501, 1439, 714, 598, 591, 1670, 721, 244, 1270, 590, 1339, 1418, 1630, 1552, 682, 611, 1010, 976, 109, 1443, 752, 1402, 1629, 716, 360, 490, 1730, 1239, 1722, 1240, 1359, 200, 1098, 647, 1626, 1659, 1596, 1474, 949, 491, 538, 1366, 1420, 1401, 205, 674, 268, 1079, 1626, 807, 256, 708, 546, 1735, 52, 1195, 1421, 299, 855, 360, 1630, 98, 813, 793, 1404, 489, 220, 679, 1127, 20, 1437, 417, 252, 186, 52, 292, 1146, 1104, 415, 1562, 45, 215, 593, 503, 1243, 545, 875, 1490, 308, 1046, 426, 1641, 1085, 957, 1100, 356, 748, 1625, 1110, 1602, 887, 1474, 37, 1703, 500, 1778, 1693, 199, 772, 1263, 912, 933, 26, 846, 928, 655, 353, 304, 1611, 1456, 1784, 1342, 835, 725, 228, 697, 1485, 1264, 980, 936, 1651, 1108, 1740, 

(0, (700, 10, 10))

## <center> PROCESS</center>

In [88]:
%%time

# instancia de la cola para adicionar los resultados
listado_instancias = []

# instancia de la cola para adicionar los resultados
almacenar_resultados_proceso = cola_proceso()

@almacenar_en_cola_proceso
def dormir_sincrono(tiempo:int) -> int:    
    sleep(tiempo)
    return random.randint(0,limite_tareas)
        

for tiempo_espera in range(limite_tareas):
  
    instancia = Process(name=f'ejecucion de tarea # {tiempo_espera}', target=dormir_sincrono, args=(2,))
    instancia.start()
    
    listado_instancias.append(instancia)

    
for tipo_instancia in listado_instancias:    
        tipo_instancia.join()

resultado = obtener_resultado_cola_tarea_while(almacenar_resultados_proceso)
print(len(resultado))
print(resultado)

del listado_instancias, almacenar_resultados_proceso, instancia, resultado
garbage_colector.collect(), garbage_colector.get_threshold()

1800
[324, 210, 972, 1046, 1093, 766, 1535, 283, 347, 1235, 440, 1530, 778, 619, 1196, 742, 584, 742, 1417, 98, 1479, 1075, 186, 546, 1558, 507, 123, 231, 790, 1739, 1708, 632, 1389, 235, 532, 1064, 743, 1141, 807, 345, 1116, 1270, 510, 123, 951, 1174, 1665, 1563, 79, 1012, 1690, 1133, 1568, 1071, 49, 1254, 593, 1170, 656, 38, 139, 194, 1192, 327, 1377, 474, 774, 109, 226, 1095, 1362, 710, 1576, 1225, 858, 1162, 111, 1346, 399, 797, 1609, 106, 1077, 1507, 675, 9, 818, 1710, 1490, 313, 364, 1716, 476, 832, 986, 244, 914, 948, 975, 647, 819, 1537, 864, 1174, 1496, 206, 818, 770, 1446, 400, 440, 595, 1641, 1703, 1342, 1544, 431, 1721, 1602, 1484, 1698, 82, 1480, 308, 1226, 1325, 1320, 1032, 364, 994, 1329, 1081, 838, 1287, 651, 69, 1082, 295, 872, 1744, 448, 1394, 889, 679, 993, 394, 1278, 1157, 696, 1709, 1108, 65, 586, 1350, 473, 1584, 1730, 1736, 98, 357, 1085, 847, 151, 621, 1496, 1172, 430, 1791, 150, 53, 435, 12, 946, 1254, 575, 1325, 1531, 386, 1586, 1383, 609, 1072, 1733, 1354, 19

(0, (700, 10, 10))

## <center> **POOL PROCESS DE MULTIPROCESO**</center>

In [89]:
%%time

def dormir_sincrono_multiprocessing(contador:int) -> int:    
    sleep(2)
    return random.randint(0,limite_tareas)

with Pool() as multiprocesing:
    pool = multiprocesing.map(dormir_sincrono_multiprocessing, range(limite_tareas//3))
print(list(pool))

del multiprocesing, pool
garbage_colector.collect(), garbage_colector.get_threshold()

[996, 827, 1488, 1156, 1452, 1761, 1615, 1614, 472, 929, 466, 1668, 134, 1344, 22, 649, 1568, 882, 581, 1138, 167, 498, 1693, 576, 1726, 650, 842, 991, 394, 51, 819, 378, 132, 279, 554, 737, 1600, 825, 1799, 1379, 51, 908, 1788, 1476, 644, 1273, 1088, 1003, 223, 89, 1409, 1704, 677, 351, 1366, 687, 434, 1593, 1572, 1310, 1549, 1587, 791, 875, 1000, 1201, 1583, 561, 1718, 831, 762, 327, 412, 72, 1484, 1139, 1700, 68, 649, 1369, 201, 991, 161, 1769, 944, 1118, 791, 1642, 1058, 1591, 643, 408, 76, 160, 547, 22, 1024, 1148, 1790, 1481, 793, 1287, 45, 334, 388, 1499, 1714, 280, 303, 983, 1349, 569, 1331, 1671, 1016, 1173, 987, 271, 661, 1283, 159, 62, 1426, 1143, 37, 117, 808, 58, 1584, 507, 395, 536, 66, 833, 485, 1453, 1701, 1497, 1735, 1186, 82, 108, 861, 106, 456, 1215, 340, 173, 390, 597, 1086, 1096, 1578, 1499, 272, 958, 648, 1431, 484, 1129, 1510, 59, 1560, 620, 896, 701, 926, 371, 1168, 796, 1462, 196, 895, 1626, 1050, 1456, 1719, 383, 1488, 414, 581, 176, 1267, 83, 180, 754, 843, 5

(0, (700, 10, 10))

## <center> **THREAD POOL DE MULTIPROCESO**</center>

In [90]:
%%time

with ThreadPool() as multiprocesing:
    pool = multiprocesing.map(dormir_sincrono_multiprocessing, range(limite_tareas//3))
print(list(pool))

del multiprocesing, pool
garbage_colector.collect(), garbage_colector.get_threshold()

[504, 1374, 856, 1593, 1693, 56, 1549, 3, 1587, 1114, 551, 1151, 1045, 349, 1450, 1397, 1595, 1608, 707, 1125, 1599, 803, 800, 1738, 273, 674, 921, 1568, 759, 364, 870, 1335, 623, 1395, 1733, 573, 839, 1684, 1407, 1563, 386, 1666, 534, 1683, 26, 97, 1103, 77, 1196, 1400, 1591, 761, 1009, 1407, 1485, 1419, 766, 340, 765, 391, 639, 551, 1607, 272, 531, 830, 1699, 919, 1370, 684, 206, 6, 1773, 1716, 1544, 85, 1188, 731, 1407, 1308, 738, 1514, 251, 1360, 1093, 146, 1107, 886, 1760, 520, 1304, 716, 202, 1566, 1262, 1198, 1547, 1420, 558, 1456, 81, 191, 407, 1018, 174, 732, 577, 233, 1487, 707, 390, 330, 645, 965, 262, 1106, 992, 77, 1293, 12, 283, 519, 964, 85, 1478, 609, 1726, 187, 275, 1040, 1197, 1215, 1784, 1085, 1632, 827, 806, 827, 1025, 1634, 502, 1232, 1127, 188, 1343, 734, 676, 1741, 1796, 1400, 403, 1775, 1515, 786, 1334, 1495, 44, 1635, 338, 1476, 1058, 577, 912, 1106, 575, 1290, 464, 1249, 187, 705, 786, 650, 1588, 1043, 46, 662, 954, 1105, 525, 1639, 787, 1072, 1193, 603, 1346,

(0, (700, 10, 10))

## <center> **PROCESS POOL EXECUTOR DE CONCURRENT.FUTURES**</center>

In [91]:
%%time

with ProcessPoolExecutor() as executor:
    pool = executor.map(dormir_sincrono_multiprocessing, range(limite_tareas//3))
print(list(pool))

del executor, pool
garbage_colector.collect(), garbage_colector.get_threshold()

[1297, 546, 320, 1201, 996, 761, 1277, 1152, 1359, 1513, 792, 1501, 681, 1178, 645, 1330, 1289, 1175, 479, 56, 1799, 968, 1415, 1710, 732, 1490, 1410, 77, 302, 318, 212, 1294, 1762, 1168, 908, 1335, 483, 1698, 1281, 333, 1590, 1049, 1425, 3, 894, 1159, 196, 569, 542, 116, 53, 889, 269, 1080, 1220, 812, 1167, 242, 907, 1039, 699, 927, 1053, 1211, 412, 273, 1711, 1392, 968, 1082, 593, 1739, 769, 510, 1116, 26, 1136, 62, 362, 781, 1162, 307, 1181, 376, 1010, 468, 296, 720, 341, 682, 886, 216, 588, 512, 729, 1288, 1724, 462, 1257, 742, 1212, 1146, 26, 514, 1601, 978, 427, 653, 1389, 1639, 1444, 1519, 74, 1097, 679, 580, 948, 293, 499, 1328, 1234, 882, 254, 926, 358, 811, 910, 596, 1225, 350, 1631, 957, 1413, 255, 1760, 1483, 1556, 1650, 437, 504, 24, 1059, 706, 43, 1743, 955, 1010, 1260, 957, 978, 1601, 1023, 952, 567, 741, 1094, 460, 769, 1334, 16, 103, 147, 1436, 880, 724, 209, 566, 692, 1447, 730, 1685, 1484, 1404, 250, 611, 728, 1284, 1572, 1311, 384, 1616, 1096, 758, 791, 1436, 1288, 

(0, (700, 10, 10))

## <center> **THREAD POOL EXECUTOR DE CONCURRENT.FUTURES**</center>

In [92]:
%%time

with ThreadPoolExecutor() as executor:
    pool = executor.map(dormir_sincrono_multiprocessing, range(limite_tareas//3))
print(list(pool))

del executor, pool
garbage_colector.collect(), garbage_colector.get_threshold()

[721, 1264, 1027, 1448, 857, 1272, 1322, 1152, 908, 1792, 670, 543, 973, 791, 39, 86, 249, 982, 717, 1660, 1268, 636, 1264, 1141, 468, 804, 661, 1283, 1394, 1581, 1547, 986, 822, 892, 1471, 1299, 601, 688, 751, 1247, 1097, 434, 727, 823, 1518, 937, 145, 309, 303, 922, 188, 1028, 1277, 66, 1707, 1187, 304, 1455, 1038, 1767, 1704, 1388, 1071, 650, 453, 1086, 338, 450, 1544, 716, 471, 800, 1058, 1597, 1662, 1133, 1456, 1748, 489, 884, 1276, 1530, 1607, 1320, 1725, 80, 524, 1166, 748, 726, 312, 445, 1019, 291, 562, 410, 1394, 925, 1257, 404, 1744, 417, 1625, 1380, 969, 168, 548, 666, 794, 558, 497, 848, 504, 927, 485, 1372, 206, 205, 357, 896, 1243, 70, 1393, 299, 1345, 184, 1441, 1271, 1413, 1555, 1259, 554, 1186, 1699, 230, 190, 1773, 1478, 998, 1334, 1131, 778, 1653, 1719, 918, 1764, 1634, 420, 514, 1338, 137, 795, 1581, 726, 1790, 748, 125, 1051, 786, 1040, 1768, 1515, 1212, 957, 1396, 188, 386, 754, 165, 1353, 897, 135, 1750, 713, 1684, 728, 1411, 1045, 1073, 1332, 1056, 1505, 625, 55

(0, (700, 10, 10))

## <center> **COMPARTIR DATOS ENTRE HILOS Y PROCESOS**</center>

## <center> Compartir Datos Globales entre Hilos. 

In [93]:
%%time

compartir = 0
limite_hilos = 4
limite_iterador = 25

def memoria_compartida():
    
    global compartir    
    
    compartir_local  = compartir    
    
    for i in range(limite_iterador):
        compartir_local += 1
        
    sleep(0.2)
    
    for i in range(limite_iterador):
        compartir_local += 1
       
    compartir  = compartir_local
    
    print(f'Valor parcial {compartir}')

hilos = []

for hilo_numero in range(limite_hilos):
    
    hilo = Thread(target=memoria_compartida)
    hilo.start() 
    
    hilos.append(hilo)
    
for instancia_hilo in hilos:
    
    instancia_hilo.join()
    

print(f'Valor final {compartir}')

del compartir, limite_hilos, limite_iterador
garbage_colector.collect(), garbage_colector.get_threshold()

Valor parcial 50
Valor parcial 50
Valor parcial 50
Valor parcial 50
Valor final 50
CPU times: user 62.2 ms, sys: 0 ns, total: 62.2 ms
Wall time: 258 ms


(0, (700, 10, 10))

## <center> Definiendo la seccion critica. 

In [94]:
%%time

from threading import Lock
# instancia que bloquea hasta que finalice la seccion critica
seccion_critica = Lock()

compartir = 0
limite_hilos = 4
limite_iterador = 25

def memoria_compartida():
    
    global compartir    
    
    # debe estar ANTES de asignar el valor de la variable, que es el INICIO de la seccion critica
    seccion_critica.acquire()
    
    compartir_local  = compartir    
    
    for i in range(limite_iterador):
        compartir_local += 1
        
    sleep(0.2)
    
    for i in range(limite_iterador):
        compartir_local += 1
       
    compartir  = compartir_local
    
    # debe estar DESPUES de asignar el valor de la variable, que es el FINAL de la seccion critica
    seccion_critica.release()
    
    print(f'Valor parcial {compartir}')

hilos = []

for hilo_numero in range(limite_hilos):
    
    hilo = Thread(target=memoria_compartida)
    hilo.start()
    
    hilos.append(hilo)
    
for instancia_hilo in hilos:
    
    instancia_hilo.join()
    

print(f'Valor final {compartir}')

del seccion_critica, compartir, limite_hilos, limite_iterador
garbage_colector.collect(), garbage_colector.get_threshold()

Valor parcial 50
Valor parcial 100
Valor parcial 150
Valor parcial 200
Valor final 200
CPU times: user 165 ms, sys: 0 ns, total: 165 ms
Wall time: 958 ms


(0, (700, 10, 10))

## <center> Compartir Datos entre Procesos. 

In [95]:
%%time

compartir = 0
limite_procesos = 4
limite_iterador = 25
    
def datos_compartidos():
    
    global compartir
    
    compartir_local  = compartir
    
    for i in range(limite_iterador):
        compartir_local += 1
        
    sleep(0.2)
    
    for i in range(limite_iterador):
        compartir_local += 1
     
    compartir  = compartir_local
    print(f'Valor parcial {compartir}')

procesos = []

for hilo_numero in range(limite_procesos):
    
    proceso = Process(target=datos_compartidos)
    proceso.start()
    
    procesos.append(proceso)
    
for instancia_proceso in procesos:
    
    instancia_proceso.join()
    

print(f'Valor final {compartir}')

del compartir, limite_procesos, limite_iterador
garbage_colector.collect(), garbage_colector.get_threshold()

Valor parcial 50
Valor parcial 50
Valor parcial 50
Valor parcial 50
Valor final 0
CPU times: user 84.7 ms, sys: 116 ms, total: 200 ms
Wall time: 385 ms


(0, (700, 10, 10))

## <center> Sin definir la seccion critica. 

In [96]:
%%time
from multiprocessing import Value

# instancia que permite compartir los valores, definiendo el tipo de dato int
compartir = Value('i',0)

limite_procesos = 4
limite_iterador = 25

def datos_compartidos(): 
    
    global compartir
    
    for i in range(limite_iterador):
        compartir.value += 1
        
    sleep(0.2)
    
    for i in range(limite_iterador):
        compartir.value += 1
    
    print(f'Valor parcial {compartir.value}')

procesos = []

for hilo_numero in range(limite_procesos):
    
    proceso = Process(target=datos_compartidos)
    proceso.start()
    
    procesos.append(proceso)
    
for instancia_proceso in procesos:
    
    instancia_proceso.join()
    

print(f'Valor final {compartir.value}')

del compartir, limite_procesos, limite_iterador
garbage_colector.collect(), garbage_colector.get_threshold()

Valor parcial 125
Valor parcial 150
Valor parcial 175
Valor parcial 200
Valor final 200
CPU times: user 110 ms, sys: 65.6 ms, total: 176 ms
Wall time: 378 ms


(0, (700, 10, 10))

## <center> Definiendo la seccion critica. 

In [97]:
%%time
from multiprocessing import Lock

# instancia que bloquea hasta que finalice la seccion critica
seccion_critica = Lock()

# instancia que permite compartir los valores
compartir = Value('i',0)

limite_procesos = 4
limite_iterador = 25

def datos_compartidos():
    
    global compartir
    
    # debe estar ANTES de asignar el valor de la variable, que es el INICIO de la seccion critica
    seccion_critica.acquire() 
    
    for i in range(limite_iterador):
        compartir.value += 1
        
    sleep(0.2)
    
    for i in range(limite_iterador):
        compartir.value += 1
    
    # debe estar DESPUES de asignar el valor de la variable, que es el FINAL de la seccion critica
    seccion_critica.release()
    
    print(f'Valor parcial {compartir.value}')

procesos = []

for hilo_numero in range(limite_procesos):
    
    proceso = Process(target=datos_compartidos)
    proceso.start()
    
    procesos.append(proceso)
    
for instancia_proceso in procesos:
    
    instancia_proceso.join()
    

print(f'Valor final {compartir.value}')

del seccion_critica, compartir, limite_procesos, limite_iterador
garbage_colector.collect(), garbage_colector.get_threshold()

Valor parcial 50
Valor parcial 100
Valor parcial 150
Valor parcial 200
Valor final 200
CPU times: user 181 ms, sys: 82.5 ms, total: 263 ms
Wall time: 1.02 s


(0, (700, 10, 10))

## <center> Compartir Datos entre THREAD POOL. 
### <center> SIN SECCION CRITICA. 

In [98]:
# instancia que permite compartir los valores
compartir = Value('i',0)
seccion_critica = Lock()

def datos_compartidos(incremento): 
    
    compartir.value += incremento
    sleep(0.2)    
    print(f'Valor parcial {compartir.value}')
    
    return compartir.value
 
with ThreadPool() as pool_de_hilos:
    multihilo = pool_de_hilos.map(datos_compartidos, range(1, LIMITE))

print(multihilo)
del multihilo, pool_de_hilos, compartir, seccion_critica
garbage_colector.collect(), garbage_colector.get_threshold()

Valor parcial 10
Valor parcial 15
Valor parcial 21
Valor parcial 28
Valor parcial 36
Valor parcial 45
Valor parcial 55
Valor parcial 55
Valor parcial 55
Valor parcial 55
[10, 28, 15, 21, 36, 45, 55, 55, 55, 55]


(0, (700, 10, 10))

### <center> CON SECCION CRITICA. 

In [99]:
# instancia que permite compartir los valores
compartir = Value('i',0)
seccion_critica = Lock()

def datos_compartidos(incremento): 
    
    seccion_critica.acquire()
    compartir.value += incremento
    sleep(0.2)
    
    print(f'Valor parcial {compartir.value}') # acceder al valor antes de retornarlo puede interferir en el valor final retornado
    seccion_critica.release()   
    
    
    return compartir.value
    
with ThreadPool() as pool_de_hilos:
    multihilo = pool_de_hilos.map(datos_compartidos, range(1, LIMITE))

print(multihilo)
del multihilo, pool_de_hilos, compartir, seccion_critica
garbage_colector.collect(), garbage_colector.get_threshold()

Valor parcial 1
Valor parcial 3
Valor parcial 6
Valor parcial 10
Valor parcial 15
Valor parcial 24
Valor parcial 31
Valor parcial 39
Valor parcial 45
Valor parcial 55
[1, 3, 6, 10, 15, 45, 31, 39, 24, 55]


(0, (700, 10, 10))

## <center> Compartir Datos entre POOL de Procesos. 
## <center> Sin seccion critica. 

In [100]:
%%time
# instancia que permite compartir los valores
compartir = Value('i',0)
seccion_critica = Lock()

def datos_compartidos(incremento):
    
    global compartir
    
    compartir.value += incremento
    sleep(0.2)
    
    print(f'Valor parcial {compartir.value}')
    
    return compartir.value
    
with Pool() as pool_de_procesos:
    multiproceso = pool_de_procesos.map(datos_compartidos, range(1, LIMITE))
    
print(multiproceso)

del multiproceso, pool_de_procesos, compartir, seccion_critica
garbage_colector.collect(), garbage_colector.get_threshold()

Valor parcial 10Valor parcial 10Valor parcial 10

Valor parcial 10

Valor parcial 36
Valor parcial 36Valor parcial 36Valor parcial 45


Valor parcial 55
Valor parcial 55
[21, 10, 15, 28, 36, 45, 55, 55, 55, 55]
CPU times: user 158 ms, sys: 105 ms, total: 263 ms
Wall time: 816 ms


(0, (700, 10, 10))

## <center> Definiendo la seccion critica. 

In [101]:
# instancia que permite compartir los valores
compartir = Value('i',0)
seccion_critica = Lock()

def datos_compartidos(incremento):
    
    global compartir
    
    seccion_critica.acquire()
    
    compartir.value += incremento
    sleep(0.2)
    print(f'Valor parcial {compartir.value}')
    seccion_critica.release()  
      
    return compartir.value
    
    
with Pool() as pool_de_procesos:
    multiproceso = pool_de_procesos.map(datos_compartidos, range(1, LIMITE))
    
print(multiproceso)
del multiproceso, pool_de_procesos, compartir, seccion_critica
garbage_colector.collect(), garbage_colector.get_threshold()

Valor parcial 1
Valor parcial 3
Valor parcial 6
Valor parcial 10
Valor parcial 15
Valor parcial 21
Valor parcial 28
Valor parcial 36
Valor parcial 45
Valor parcial 55
[1, 3, 6, 10, 15, 21, 28, 36, 45, 55]


(0, (700, 10, 10))

## <center> Definiendo la seccion critica. **SIN PRINT**, OBTIENE LOS VALORES DE RETORNO CORRECTAMENTE, POR NO ACCEDER A ELLOS

In [102]:
# instancia que permite compartir los valores
compartir = Value('i',0)
seccion_critica = Lock()

def datos_compartidos(incremento):
    
    global compartir
    
    seccion_critica.acquire()
    
    compartir.value += incremento
    sleep(0.2)
    seccion_critica.release()  
      
    return compartir.value
    
    
with Pool() as pool_de_procesos:
    multiproceso = pool_de_procesos.map(datos_compartidos, range(1, LIMITE))
    
print(multiproceso)
del multiproceso, pool_de_procesos, compartir, seccion_critica
garbage_colector.collect(), garbage_colector.get_threshold()

[1, 3, 6, 10, 15, 21, 28, 36, 45, 55]


(0, (700, 10, 10))

# <center> POOL DE PROCESOS E HILOS OTROS METODOS</center>
## <center>Ofrece un medio conveniente de paralelizar la ejecución de una función a través de múltiples valores de entrada, distribuyendo los datos de entrada a través de procesos (paralelismo de datos).</center>

In [103]:
from multiprocessing.pool import Pool

def calcular_cuadrado(x):
    return x*x

### <center> APPLY - PROCESOS</center>

In [104]:
%%time
pool = Pool()
pool_apply = pool.apply(calcular_cuadrado, [3])
pool.close() # Prevents any more tasks from being submitted to the pool. Once all the tasks have been completed the worker processes will exit.
pool.join() # Wait for the worker processes to exit. One must call close() or terminate() before using join().
pool.close() # Close the Process object, releasing all resources associated with it.
pool_apply

print(pool_apply)
del pool_apply, pool
garbage_colector.collect(), garbage_colector.get_threshold()

9
CPU times: user 78.4 ms, sys: 52.9 ms, total: 131 ms
Wall time: 134 ms


(0, (700, 10, 10))

### <center> APPLY - HILOS</center>

In [105]:
%%time
pool = ThreadPool()
pool_apply = pool.apply(calcular_cuadrado, [4])
pool.close() # Prevents any more tasks from being submitted to the pool. Once all the tasks have been completed the worker processes will exit.
pool.join() # Wait for the worker processes to exit. One must call close() or terminate() before using join().
pool.close() # Close the Process object, releasing all resources associated with it.
pool_apply

print(pool_apply)
del pool_apply, pool
garbage_colector.collect(), garbage_colector.get_threshold()

16
CPU times: user 60.2 ms, sys: 10.3 ms, total: 70.5 ms
Wall time: 69.9 ms


(0, (700, 10, 10))

### <center> APPLY_ASYNC</center>

In [106]:
%%time
with Pool() as pool:
    values = pool.apply_async(calcular_cuadrado, [5])

print(values.get())
del pool, values
garbage_colector.collect(), garbage_colector.get_threshold()

25
CPU times: user 70.1 ms, sys: 64.8 ms, total: 135 ms
Wall time: 142 ms


(0, (700, 10, 10))

### <center> MAP_ASYNC</center>

In [107]:
%%time
with Pool() as pool:
    values = pool.map_async(calcular_cuadrado, [6])

print(values.get())
del pool, values
garbage_colector.collect(), garbage_colector.get_threshold()

[36]
CPU times: user 82.2 ms, sys: 95.4 ms, total: 178 ms
Wall time: 167 ms


(0, (700, 10, 10))

### <center> MAP</center>

In [108]:
%%time
with Pool() as pool:
    values = pool.map(calcular_cuadrado, range(0,20,5))

print(values)
del pool, values
garbage_colector.collect(), garbage_colector.get_threshold()

[0, 25, 100, 225]
CPU times: user 125 ms, sys: 97 ms, total: 222 ms
Wall time: 216 ms


(0, (700, 10, 10))

### <center> IMAP</center>

In [109]:
%%time
with Pool() as pool:
    values = pool.imap(calcular_cuadrado, range(0,20,4))

print(values)
del pool, values
garbage_colector.collect(), garbage_colector.get_threshold()

CPU times: user 66.4 ms, sys: 75.2 ms, total: 142 ms
Wall time: 148 ms


(151, (700, 10, 10))

### <center> IMAP_UNORDERED</center>

In [110]:
%%time
with Pool() as pool:
    values = pool.imap_unordered(calcular_cuadrado, range(0,20,2))

print(values)
del pool, values
garbage_colector.collect(), garbage_colector.get_threshold()

CPU times: user 61 ms, sys: 63.9 ms, total: 125 ms
Wall time: 126 ms


(152, (700, 10, 10))

### <center> STARMAP</center>

In [111]:
%%time
with Pool() as pool:
    values = pool.starmap(calcular_cuadrado, [[1],[2],[3],[4],[5]])

print(values)
del pool, values
garbage_colector.collect(), garbage_colector.get_threshold()

[1, 4, 9, 16, 25]
CPU times: user 41.2 ms, sys: 84.5 ms, total: 126 ms
Wall time: 126 ms


(0, (700, 10, 10))

### <center> STARMAP_ASYNC</center>

In [112]:
%%time
with Pool() as pool:
    values = pool.starmap_async(calcular_cuadrado, [[1],[2],[3],[4],[5]])

print(values)
del pool, values
garbage_colector.collect(), garbage_colector.get_threshold()

CPU times: user 75.3 ms, sys: 52.6 ms, total: 128 ms
Wall time: 129 ms


(153, (700, 10, 10))

# <center> OTRAS **FUNCIONES ASINCRONAS**</center>

In [113]:
async def Primero(limite=5):
    print("Inicio funcion Primero")
    await asyncio.sleep(2)
    print("Final funcion Primero")
    
    return [random.randrange(-100, 0) for x in range(limite)]

async def Ultimo(limite=5):
    print("Comenzar funcion ultimo")
    await asyncio.sleep(2)
    print("Terminar funcion ultimo")
    
    return [random.randrange(0, 100) for x in range(limite)]

# <center> **GATHER**</center>
## <center> **RECIBE UNA LISTA DE TAREAS Y RETORNA UNA LISTA DE RESULTADOS**</center>
## <center> **TIENE OPCIONES ESPECÍFICAS PARA EL MANEJO DE ERRORES Y CANCELACIONES.**</center>

In [114]:
# El siguiente ejemplo muestra cómo esperar a que se completen varias tareas asincrónicas.

from asyncio import gather

async def iniciar_gather():
    
    tareas = [Primero(), Ultimo(), Primero(), Ultimo(), Primero(), Ultimo(), Primero()]
    
    return await gather(*tareas)
    
await iniciar_gather()

Inicio funcion Primero
Comenzar funcion ultimo
Inicio funcion Primero
Comenzar funcion ultimo
Inicio funcion Primero
Comenzar funcion ultimo
Inicio funcion Primero
Final funcion Primero
Terminar funcion ultimo
Final funcion Primero
Terminar funcion ultimo
Final funcion Primero
Terminar funcion ultimo
Final funcion Primero


[[-80, -51, -14, -99, -64],
 [96, 62, 40, 60, 21],
 [-54, -35, -50, -58, -1],
 [17, 76, 64, 59, 83],
 [-89, -98, -77, -16, -33],
 [62, 44, 42, 23, 90],
 [-7, -69, -98, -57, -37]]

# <center> **WAIT_FOR**</center>
## <center> **PERMITE DEFINIR EL MAXIMO TIEMPO DE ESPERA DE UNA TAREA**</center>

In [115]:
# El siguiente ejemplo demuestra cómo podemos utilizar un tiempo de espera para evitar esperar indefinida a que finalice una tarea asincrónica.

from asyncio import wait_for

async def iniciar_wait_for():
    try:
        return await wait_for(Primero(), timeout=1)
    except asyncio.TimeoutError:
        print("la ejecucion supero el tiempo de espera!")
        

await iniciar_wait_for()

Inicio funcion Primero
la ejecucion supero el tiempo de espera!


# <center> **AS_COMPLETED**</center>
## <center> **ES SIMILIAR A GATHER, PERO RETORNA FUTUROS, LOS RESULTADOS SON RETORNADOS EN EL ORDEN QUE ESTAN LISTOS**</center>

In [116]:
# El siguiente ejemplo demuestra cómo as_complete, completará la primera tarea, seguida de la siguiente más rápida y la siguiente hasta que se completen todas las tareas.

from asyncio import as_completed

async def iniciar_as_completed():
    
    tareas = [Primero(), Ultimo(), Primero(), Ultimo(), Primero(), Ultimo(), Primero()]
    counter = 0
    
    for future in as_completed(tareas):
        n = "la tarea mas rapida" if counter == 0 else "la siguiente tarea mas rapida"
        counter += 1
        result = await future
        print(f"{n} obtuvo el resultado: {result}")

await iniciar_as_completed()

Inicio funcion Primero
Comenzar funcion ultimo
Comenzar funcion ultimo
Inicio funcion Primero
Inicio funcion Primero
Comenzar funcion ultimo
Inicio funcion Primero
Final funcion Primero
Terminar funcion ultimo
Terminar funcion ultimo
Final funcion Primero
Final funcion Primero
Terminar funcion ultimo
Final funcion Primero
la tarea mas rapida obtuvo el resultado: [-69, -8, -32, -30, -32]
la siguiente tarea mas rapida obtuvo el resultado: [52, 87, 94, 8, 46]
la siguiente tarea mas rapida obtuvo el resultado: [67, 97, 23, 90, 92]
la siguiente tarea mas rapida obtuvo el resultado: [-34, -20, -58, -84, -50]
la siguiente tarea mas rapida obtuvo el resultado: [-8, -45, -9, -71, -44]
la siguiente tarea mas rapida obtuvo el resultado: [27, 95, 12, 6, 6]
la siguiente tarea mas rapida obtuvo el resultado: [-39, -97, -57, -100, -7]


# <center> **CREATE_TASK**</center>

In [117]:
# El siguiente ejemplo demuestra cómo convertir una rutina en una tarea y programarla en el bucle de eventos.

from asyncio import create_task

async def iniciar_create_task():
    
    tarea = create_task(Primero())
    print(tarea)
    
    await asyncio.sleep(2)
    print("MAS PROCESOS!")
    
    await asyncio.sleep(3)
    print(sum(tarea))
    
    return tarea

await iniciar_create_task()

<Task pending name='Task-1835' coro=<Primero() running at /tmp/ipykernel_657/4294725927.py:1>>
Inicio funcion Primero
MAS PROCESOS!
Final funcion Primero
0


<Task finished name='Task-1835' coro=<Primero() done, defined at /tmp/ipykernel_657/4294725927.py:1> result=[-1, -39, -75, -72, -9]>

# <center> **POOLS**</center>
# <center> **GET_RUNNING_LOOP**</center>

## <center> **ASYNCIO Event Loop - MULTI HILO**</center>

In [118]:
# importa la libtreria
from asyncio import get_running_loop
# importa librerias para trabajar concurrencia
from concurrent.futures import ThreadPoolExecutor

def blocking_io(value = 23, otros = 10):
    # File operations (such as logging) can block the
    # event loop: run them in a thread pool.
    with open("/dev/urandom", "rb") as f:
        return f.read(value + otros)
    
async def asyncio_multi_hilo():
    
    loop = get_running_loop()
    pool = ThreadPoolExecutor()
    
    # tipo de pool, funcion, parametros
    return await loop.run_in_executor(pool, blocking_io, 50, 70)

await asyncio_multi_hilo()

b'\xe2\xb9\xda~\xe8\xb3\r3\xa4\xe00\x05!\xdd]\x14J=\x95\r;\xe4&\x1ae\x16\x0f\xf7W~o \xfb5\x830L\xa4y7\x10anS\xa6\x17z\xeb\x80\x94N\x1ee\xa8\x0e\x97\xb3\xc9Q\xff`?\xe8\xbd\x80\xef\xfd\xd3\xdb_.\x9c\x07\xf2\x1c\xa6q\x04F\xe0\xa5\x96Zt\x87_n\x88P8n[G\r\x1cN\xd7\xb2\xde\x12\x80B!\x8b%\xe6\xd4S\x0e:\xa4#\xefvUe~k,y'